In [2]:
import numpy as np

class SGD:
    def __init__(self, lr) -> None:
        self.lr = lr
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

BTS = SGD(0.05)
prams = {'a': 1, 'b': 2}
grads = {'a': 0.1, 'b': 0.2}
BTS.update(prams, grads)
print(prams) # {'a': 0.95, 'b': 1.9}

{'a': 0.995, 'b': 1.99}


In [6]:
class Momentum:
    def __init__(self,lr,momentum) -> None:
        self.lr = lr    
        self.momentum = momentum
        self.v = None
    def update(self,params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)   
        
        for key in params.keys():
            self.v[key] = self.momentum * self.v[key] - self.lr * grads[key]
            params[key] += self.v[key]

momentum = Momentum(0.05,0.9)
prams = {'a': 1, 'b': 2}
grads = {'a': 0.1, 'b': 0.2}
momentum.update(prams, grads)
print(prams) # {'a': 0.995, 'b': 1.99}

{'a': 0.995, 'b': 1.99}


In [1]:
class AdaGrad:
    def __init__(self,lr) -> None:
        self.lr = lr
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)
            
ada = AdaGrad(0.01)
prams = {'a': 1.0, 'b': 2.0}
grads = {'a': 0.1, 'b': 0.2}
ada.update(prams, grads)
print(prams) # {'a': 0.9999999999999999, 'b': 1.9999999999999998}

NameError: name 'np' is not defined

레포트는 옵티마이저 비교하는 것 코드, 그래프 출력까지한걸 올려야함 이번주까지

In [3]:
import numpy as np

def c_c_t(corpus, window_size=1):
    # 타겟 단어들은 중심 단어들로, 앞뒤 윈도우 크기만큼의 단어들을 제외한 부분이다.
    target = corpus[window_size:-window_size]
    
    # 컨텍스트 단어들을 담을 리스트를 초기화한다.
    contexts = []

    # 중심 단어의 인덱스를 순회한다.
    for idx in range(window_size, len(corpus) - window_size):
        # 현재 중심 단어의 컨텍스트 단어들을 담을 리스트를 초기화한다.
        cs = []
        
        # 중심 단어의 양옆 윈도우 크기만큼의 단어들을 순회한다.
        for t in range(-window_size, window_size + 1):
            # t가 0인 경우는 중심 단어이므로 건너뛴다.
            if t == 0:
                continue
            
            # 컨텍스트 단어를 리스트에 추가한다.
            cs.append(corpus[idx + t])
        
        # 현재 중심 단어의 컨텍스트 단어 리스트를 contexts 리스트에 추가한다.
        contexts.append(cs)

    # contexts와 target을 넘파이 배열로 변환하여 반환한다.
    return np.array(contexts), np.array(target)

# 예제 문장
corpus = ["The", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog"]
# 윈도우 크기 설정
window_size = 2
# 함수 호출
contexts, target = c_c_t(corpus, window_size)

# 결과 출력
print("Contexts:\n", contexts)
print("Target:\n", target)


Contexts:
 [['The' 'quick' 'fox' 'jumps']
 ['quick' 'brown' 'jumps' 'over']
 ['brown' 'fox' 'over' 'the']
 ['fox' 'jumps' 'the' 'lazy']
 ['jumps' 'over' 'lazy' 'dog']]
Target:
 ['brown' 'fox' 'jumps' 'over' 'the']


In [9]:
import numpy as np

def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    col : 2차원 배열
    """
    # 입력 데이터의 형상 정보(N: 이미지 수, C: 채널 수, H: 높이, W: 너비)를 얻어온다.
    N, C, H, W = input_data.shape
    
    # 출력 높이와 너비를 계산한다.
    out_h = (H + 2*pad - filter_h) // stride + 1
    out_w = (W + 2*pad - filter_w) // stride + 1

    # 입력 데이터에 패딩을 추가한다.
    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    
    # 필터 적용을 위한 빈 배열을 생성한다.
    # shape: (N, C, filter_h, filter_w, out_h, out_w)
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))
    print("col",col.shape)
    # 필터의 높이 방향으로 순회한다.
    for y in range(filter_h):
        # 스트라이드를 고려한 y_max를 계산한다.
        y_max = y + stride * out_h
        print("y_max",y_max)
        # 필터의 너비 방향으로 순회한다.
        for x in range(filter_w):
            # 스트라이드를 고려한 x_max를 계산한다.
            x_max = x + stride * out_w
            
            # 필터 영역의 값을 col 배열에 할당한다.
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]
            print("col",col.shape)
            print("img",img.shape)
    # col 배열을 재배열(transpose)하고 2차원 배열로 변환(reshape)한다.
    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N * out_h * out_w, -1)
    
    return col

# 예제 사용법
input_data = np.random.rand(2, 3, 224, 224)  # (10개의 이미지, 3개의 채널, 28x28 크기)
filter_h = 3  # 필터 높이
filter_w = 3  # 필터 너비
stride = 1  # 스트라이드
pad = 1  # 패딩

# im2col 함수 호출
col = im2col(input_data, filter_h, filter_w, stride, pad)

print("Shape of the input data:", input_data.shape)
print("Shape of the output data:", col.shape)


col (2, 3, 3, 3, 224, 224)
y_max 224
col (2, 3, 3, 3, 224, 224)
img (2, 3, 226, 226)
col (2, 3, 3, 3, 224, 224)
img (2, 3, 226, 226)
col (2, 3, 3, 3, 224, 224)
img (2, 3, 226, 226)
y_max 225
col (2, 3, 3, 3, 224, 224)
img (2, 3, 226, 226)
col (2, 3, 3, 3, 224, 224)
img (2, 3, 226, 226)
col (2, 3, 3, 3, 224, 224)
img (2, 3, 226, 226)
y_max 226
col (2, 3, 3, 3, 224, 224)
img (2, 3, 226, 226)
col (2, 3, 3, 3, 224, 224)
img (2, 3, 226, 226)
col (2, 3, 3, 3, 224, 224)
img (2, 3, 226, 226)
Shape of the input data: (2, 3, 224, 224)
Shape of the output data: (100352, 27)
